In [1]:
from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile
import numpy as np
import scipy.signal as signal
import os
import emu.neuralynx_io as neuralynx_io
from tqdm import tqdm

In [2]:
raw_root = os.path.expanduser('~/.emu/pt1/sEEG/raw')
files = [f for f in os.listdir(raw_root) if f.startswith('CSC3')]
files = sorted(files)
files

['CSC3.ncs',
 'CSC3_0001.ncs',
 'CSC3_0002.ncs',
 'CSC3_0003.ncs',
 'CSC3_0004.ncs',
 'CSC3_0005.ncs',
 'CSC3_0006.ncs',
 'CSC3_0007.ncs',
 'CSC3_0008.ncs',
 'CSC3_0009.ncs',
 'CSC3_0010.ncs',
 'CSC3_0011.ncs',
 'CSC3_0012.ncs']

In [3]:
def gen_load_timestamps(fp_list,downsample=4):
    for fp in fp_list:
        ncs = neuralynx_io.load_ncs(os.path.join(raw_root,fp))
        yield signal.decimate(ncs['timestamp'],q=downsample)

In [4]:
def gen_load_ncs(fp_list,factor):
    for fp in fp_list:
        ncs = neuralynx_io.load_ncs(os.path.join(raw_root,fp),load_time=False)
        yield signal.decimate(ncs['data'],q=factor)

In [5]:
# ncs_timestamp = []
# t_iter = gen_load_timestamps(files,downsample=4)
# for dec_t in tqdm(t_iter):
#     ncs_timestamp.append(dec_t)

In [6]:
ncs_data = []
dat_iter = gen_load_ncs(files,4)
for d in tqdm(dat_iter,total=len(files)):
    ncs_data.append(d.astype(np.float16))

  0%|          | 0/13 [00:00<?, ?it/s]/home/elijahc/projects/emu/notebooks/src/neuralynx_io.py:78: UserWarning: Unable to parse original file path from Neuralynx header: -FileType NCS
  warnings.warn('Unable to parse original file path from Neuralynx header: ' + hdr_lines[1])
/home/elijahc/projects/emu/notebooks/src/neuralynx_io.py:130: UserWarning: Unable to parse time string from Neuralynx header: -FileVersion 3.4
  warnings.warn('Unable to parse time string from Neuralynx header: ' + time_string)
/home/elijahc/projects/emu/notebooks/src/neuralynx_io.py:130: UserWarning: Unable to parse time string from Neuralynx header: -FileUUID 10a6888b-fba5-44d8-afad-10453da6460d
  warnings.warn('Unable to parse time string from Neuralynx header: ' + time_string)
/home/elijahc/projects/emu/notebooks/src/neuralynx_io.py:92: UserWarning: Unable to parse parameter line from Neuralynx header: -ProbeName
  warnings.warn('Unable to parse parameter line from Neuralynx header: ' + line)
/home/elijahc/pro

In [7]:
dat = np.concatenate(ncs_data)

In [8]:
dat.astype(np.float16)

array([ 15.35,  19.33,  20.23, ..., -27.1 , -28.94, -45.4 ], dtype=float16)

In [11]:
np.save('/home/elijahc/.emu/pt1/sEEG/data_1/CSC3',dat)

In [10]:
asdf

NameError: name 'asdf' is not defined

In [ ]:
timestamp = np.concatenate(ncs_timestamp).astype(int)

In [ ]:
np.save(os.path.join('/home/elijahc/.emu/pt1/sEEG/data_1/timestamps_3'),timestamp)

In [ ]:
ts6 = np.load('/home/elijahc/.emu/pt1/sEEG/data_1/timestamps_6.npy')

In [ ]:
ts3 = timestamp

In [ ]:
len(ts3)

In [ ]:
np.count_nonzero((ts3 == ts6).astype(int))

In [ ]:
#a

In [ ]:
dat = np.concatenate(ncs_data)

In [ ]:
len(dat)/2000/60/60/4

In [ ]:
ncs = neuralynx_io.load_ncs(os.path.expanduser('~/.emu/pt1/sEEG/raw/CSC6.ncs'))

In [ ]:
decimated_data = signal.decimate(ncs['data'],4)

In [ ]:
len(decimated_data)

In [ ]:
start_time = datetime.fromtimestamp(ncs['time'][0]/1000/1000)

In [ ]:
nwbfile = NWBFile(session_description='demonstrate NWBFile basics',  # required
                  identifier='CSC6',  # required
                  session_start_time=start_time)

In [ ]:
ncs['data_units']

In [ ]:
from pynwb import TimeSeries
data = ncs['data']


In [ ]:
rate_ts = TimeSeries(name='channel6', data=data, unit=ncs['data_units'],rate=8000.0, starting_time=0.0)

In [ ]:
ncs['time_units']

In [ ]:
test_ts = TimeSeries(name='channel6', data=data, unit=ncs['data_units'], timestamps=ncs['timestamp']/1000/1000)

In [ ]:
nwbfile.add_acquisition(rate_ts)

In [ ]:
nwbfile.acquisition['channel6']

In [ ]:
rate_ts.rate

In [ ]:
from pynwb import NWBHDF5IO
io = NWBHDF5IO(os.path.expanduser('~/.emu/pt1/sEEG/data_1/CSC6.nwb'), mode='w')

In [ ]:
io.write(nwbfile)
io.close()